In [3]:
import pandas as pd

In [8]:
# Load the CSV files for weather and power consumption into pandas dataframes
weather_data_2019 = pd.read_csv("../Cleaning/Weather_MI_2019.csv", dtype={'STATION': 'string'})
weather_data_2020 = pd.read_csv("../Cleaning/Weather_MI_2020.csv", dtype={'STATION': 'string'})
weather_data_2021 = pd.read_csv("../Cleaning/Weather_MI_2021.csv", dtype={'STATION': 'string'})
weather_data_2022 = pd.read_csv("../Cleaning/Weather_MI_2022.csv", dtype={'STATION': 'string'})
weather_data_2023 = pd.read_csv("../Cleaning/Weather_MI_2023.csv", dtype={'STATION': 'string'})

weather_data = pd.concat([weather_data_2019, weather_data_2020, weather_data_2021, weather_data_2022, weather_data_2023], ignore_index=True)

power_plants_data = pd.read_csv("../Cleaning/MI_Plant_Location.csv")

In [10]:
# Convert the 'DATE' column to datetime format in weather_data
weather_data = weather_data[pd.to_datetime(weather_data['DATE'], errors='coerce').notna()]
weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

power_plants_data['period'] = pd.to_datetime(power_plants_data['period'], format='%Y-%m')
power_plants_data = power_plants_data[power_plants_data['period']>=pd.to_datetime('2019-01-01')]
power_plants_data = power_plants_data[power_plants_data['primeMover']=="ALL"]
power_plants_data=power_plants_data.dropna()

# Round latitude and longitude columns for better matching
weather_data['LATITUDE'] = weather_data['LATITUDE'].round(4)
weather_data['LONGITUDE'] = weather_data['LONGITUDE'].round(4)
power_plants_data['Latitude'] = power_plants_data['Latitude'].round(4)
power_plants_data['Longitude'] = power_plants_data['Longitude'].round(4)

power_plants_data.head(5)

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,total-consumption,total-consumption-units,...,NG_MW,Nuclear_MW,Crude_MW,Solar_MW,Wind_MW,Other_MW,Source,Period,Longitude,Latitude
454,2023-08-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,922648.0,MMBtu per Mcf,...,223.9,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-84.6578,42.6906
458,2023-03-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,592926.0,MMBtu per Mcf,...,223.9,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-84.6578,42.6906
462,2022-04-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,909054.0,MMBtu per Mcf,...,223.9,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-84.6578,42.6906
466,2023-10-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,547826.0,MMBtu per Mcf,...,223.9,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-84.6578,42.6906
470,2022-08-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,1004884.0,MMBtu per Mcf,...,223.9,0.0,0.0,0.0,0.0,0.0,"EIA-860, EIA-860M and EIA-923",202305.0,-84.6578,42.6906


In [11]:
# Aggregate the weather data to make it more manageable
columns_to_aggregate = ['LATITUDE','LONGITUDE','DATE', 'TEMP', 'DEWP', 'SLP',
                         'STP', 'VISIB', 'WDSP', 
                         'MXSPD', 'GUST', 'MAX', 'MIN', 'PRCP',
                         'SNDP', ]
weather_subset = weather_data[columns_to_aggregate]

# Group by the monthly period (to match the natural gas data) and aggregate weather data for the selected columns
aggregated_weather_data = weather_subset.groupby(['LATITUDE', 'LONGITUDE', pd.Grouper(key='DATE', freq='MS')]).agg(['mean']).reset_index().droplevel(1, axis=1)

aggregated_weather_data.head(5)


,LATITUDE,LONGITUDE,DATE,TEMP,DEWP,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP
0,41.2747,-85.84,2022-01-01,22.132258,14.416129,9999.9,923.938710,8.912903,6.619355,13.222581,241.948387,29.967742,14.754839,74.186129,999.9
1,41.2747,-85.84,2022-02-01,27.142857,19.717857,9999.9,953.521429,8.582143,7.207143,14.192857,161.335714,35.600000,19.850000,67.850357,999.9
2,41.2747,-85.84,2022-03-01,39.654839,29.858065,9999.9,984.245161,9.138710,7.264516,15.677419,117.974194,51.161290,31.303226,48.382258,999.9
3,41.2747,-85.84,2022-04-01,45.826667,35.430000,9999.9,984.483333,9.480000,7.633333,15.743333,22.050000,55.760000,38.300000,79.992000,999.9
4,41.2747,-85.84,2022-05-01,62.616129,50.967742,9999.9,983.735484,9.470968,5.912903,12.867742,83.425806,73.225806,53.890323,48.382258,999.9


In [12]:
# Function to find the closest weather station for each power plant
def find_nearest_weather_station(row, weather_data):
    # Filter weather stations based on the same period
    filtered_weather_data = weather_data[weather_data['DATE'] == row['period']]
    if filtered_weather_data.empty:
        return None
    
    # Calculate the Euclidean distance between the power plant and each filtered weather station
    distances = ((filtered_weather_data['LATITUDE'] - row['Latitude'])**2 + (filtered_weather_data['LONGITUDE'] - row['Longitude'])**2)**0.5
    # Find the index of the weather station with the minimum distance
    nearest_station_index = distances.idxmin()
    # Get the data for the nearest weather station
    nearest_station = filtered_weather_data.loc[nearest_station_index]
    
    return nearest_station

In [13]:
# Apply the custom function to each power plant row
closest_weather_stations = power_plants_data.apply(lambda row: find_nearest_weather_station(row, aggregated_weather_data), axis=1)

In [14]:
merged_data = pd.merge(power_plants_data, closest_weather_stations, left_index=True, right_index=True, suffixes=('_plant', '_weather'))
merged_data.head(5)

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,total-consumption,total-consumption-units,...,SLP,STP,VISIB,WDSP,MXSPD,GUST,MAX,MIN,PRCP,SNDP
454,2023-08-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,922648.0,MMBtu per Mcf,...,1014.561290,983.287097,8.909677,5.854839,12.848387,400.861290,81.670968,58.432258,0.138065,999.900000
458,2023-03-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,592926.0,MMBtu per Mcf,...,1017.906452,952.787097,8.435484,8.370968,15.396774,213.812903,46.774194,27.229032,0.097742,710.458065
462,2022-04-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,909054.0,MMBtu per Mcf,...,1015.526667,983.276667,9.073333,9.273333,16.643333,123.983333,57.620000,35.900000,0.130333,999.900000
466,2023-10-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,547826.0,MMBtu per Mcf,...,1015.554839,983.629032,9.222581,6.670968,12.351613,369.432258,63.596774,43.464516,0.153871,999.900000
470,2022-08-01,63259,Delta Energy Park,NG,Natural Gas,MI,Michigan,ALL,1004884.0,MMBtu per Mcf,...,1016.296774,985.109677,9.448387,5.664516,12.803226,433.454839,84.222581,60.877419,0.201290,999.900000


In [15]:
merged_data.to_csv("Merged_Data.csv", index=False)

In [16]:
# Sanity checks - power plants data and merged data should have the same number of rows
print(len(weather_data))
print(len(power_plants_data))
print(len(merged_data))

399190
4349
4349
